## Importar todos los modulos necesarios

In [328]:
import pandas as pd
import numpy as np
import uuid
from scipy.stats import truncnorm 

## Random Values Generators

In [332]:
# Funcion normal truncada
def generate_truncated_normal_data(min_val, max_val, std, mean, size):
    a, b = (min_val - mean) / std, (max_val - mean) / std
    data = truncnorm(a, b, loc=mean, scale=std).rvs(size)
    data = np.round(data, 3)
    return data

# Generar una serie de valores aleatorios de tipo categoria con tamanio relativo dependiente del parametro size
def generate_category_random_values(arr, size):
    return pd.Series(np.random.choice(arr, size=size))

# Generar un arreglo de valores aleatorios de tipo unique con tamanio relativo dependiente del parametro size
def generate_unique_random_values(size):
    return [uuid.uuid4().hex[:16] for _ in range(size)]

# Generar un arreglo de valores aleatorios de tipo date con tamanio relativo dependiente del parametro size
def generate_date_random_values(min, max, size):
    date_range = pd.date_range(start=min, end=max, freq='D')
    return np.random.choice(date_range, size)

# Generar un arreglo  de valores aleatorios de tipo foreign con tamanio relativo dependiente del parametro size
def generate_foreign_random_values(serie, size):
    return np.random.choice(serie.drop_duplicates(), size)

# Generar un arreglo  de valores aleatorios de tipo numeric con tamanio relativo dependiente del parametro size
def generate_numeric_random_values(min, max, std, mean, size):
    if std is not None and mean is not None:
        values = generate_truncated_normal_data(min, max, std, mean, size)
    else:
        values = np.random.uniform(min, max, size)
        values = np.round(values, 3)
    return values 

## Random Dataset Simulations Generators

In [446]:
# Generar n cantidad de registros categoricos de manera aleatoria
def get_categorical_dataset_simulated(simulation_extended, category_cols, size):
    # la columna n = 1 servira para realizar la funcion de agrupacion count en el dataframe
    simulation_extended['n'] = 1

    # Generamos la agrupacion por columnas categoricas con su respectiva funcion de agregacion count para obtener la frecuencia con la que aparecen cada agrupacion
    grouped_combination = simulation_extended.groupby(
        category_cols,
        as_index=False
    ).agg({
        'n': 'count'
    })

    # Renombramos los nombres de las columnas, en este caso seran las mismas columnas categoricas + la ultima columna n que era la relacionada con la frecuencia o conteo
    grouped_combination.columns = category_cols + ['count']

    # Agregamos otra columna extra de prob para almacenar la probabilidad con la que aparecen cada grupo de columnas categoricas
    grouped_combination['prob'] = grouped_combination['count'] / dataframe_list[4].shape[0]

    # Generamos aleatoriamente un arreglo de indices en base a las combinaciones generadas anteriormente manteniendo sus probabilidades
    rand_index = np.random.choice(
        grouped_combination.index,
        size = size,
        p = list(grouped_combination['prob'])
    )

    # Retornamos los 100k registros aleatorios ya con sus respectivos valores, no como indices nada mas
    return grouped_combination.loc[rand_index, category_cols]   

# Generar los registros de manera aleatoria para las columnas numericas
def get_numeric_column_simulated( simulated , simulation_extended , category_cols , column_name ):
    # Funcion de agregacion para obtener el min, max, mean y std por cada agrupacion de columnas categoricas
    a1 = simulation_extended.groupby(
        category_cols
        , as_index = False
    ).agg(
        {
            column_name: ["min","max","mean","std"]
        }
    )

    a1.columns = [c for c in category_cols] + ["Min", "Max", "Mean", "Std"]  # Renombra las columnas de a1

    ColumnSimulated = pd.DataFrame() # Inicializa un nuevo dataframe vacio en este caso
    
    # Recorre los indices de a1
    for i in a1.index:
        # Toma el registro completo del indice i, el registro completo son toda la fila de valores de ese indice
        rs = a1.loc[i]

        # reasigna area_subarea y crea una copia de todos los registros de simulated (que al principio era una copia de area_subarea sin la columna n) que contengan el indice i
        area_subarea = simulated.loc[i].copy()    

        # Genera los valores aleatorios en base a su resumen estadistico de min, max, mean y std
        data = generate_truncated_normal_data(
            rs["Min"] - 1 if rs["Std"] == 0 else rs["Min"],
            rs["Max"] + 1 if rs["Std"] == 0 else rs["Max"],
            1 if rs["Std"] == 0 else rs["Std"],
            rs["Mean"], 
            area_subarea.shape[0]
        )

        # Agregamos la nueva columna a area_subarea
        area_subarea[column_name] = data

        # Concatenamos el conjunto de registros anteriores con el registro actual recorrido del ciclo
        ColumnSimulated = pd.concat( [ ColumnSimulated , area_subarea  ] )

    return ColumnSimulated.reset_index(drop=True) # Retornamos el resultado final restableciendo los indices
    

## Helpers

In [455]:
# Funcion encargada de calcular la columna con mayor cantidad de registros (Esto en caso de que el valor random sea False)
def calc_greater(cols_arr):
    max_values = 0 # Inicia en cero suponiendo que no hay nada todavia al inicio
    
    for col in cols_arr:
        col_type = col['type']

        if col_type == 'foreign':
            pass
        
        if col_type == 'date':
            max_values = max(max_values, pd.date_range(start=col['values']['min'], end=col['values']['max']).size)

        if col_type == 'category':
            max_values = max(max_values, len(col['values']))

    return max_values

# Funcion encargada de evaluar que en efecto las columnas relacionadas existan asi como sus posibles columnas
def validate_relationships(all_dfs, reference_df):
    # Primero valido que exista el dataframe
    df_name, df_column = reference_df.split('.')

    # Validar por nombre del dataframe
    if df_name not in all_dfs:
        raise ValueError(f'El dataframe {df_name} no existe :(')

    # Sacamos las columnas validas
    columns = list(all_dfs[df_name].columns)

    # Validamos por columna
    if df_column not in columns:
        raise ValueError(f'La columna {df_column} no existe :( en el dataframe {df_name}. Las columnas validas son: {columns}.')

# Funcion que retorna un arreglo con todas las dependencias de un objeto que contiene alguna configuracion
def extract_dependencies(obj):
    dependencies = []
    
    for col in obj['columns']:
        if col['type'] == 'foreign':
            df_name = col['values'].split('.')[0]
            dependencies.append(df_name)
    return dependencies

# Funcion encargada de reordenar por dependencias el arreglo conf_list que puede que su orden no sea el de acuerdo a sus dependencias
def reorder_by_dependencies(arr): # Usando el algoritmo de insertion sort
    for i, value in enumerate(arr[1:], start=1):
        value_dependencies = extract_dependencies(value)
        j = i - 1

        # Validar que j no sea menor que cero y que las dependencias del valor actual recorrido esten en el valor de la izq
        while j >= 0 and any(dep in [d['ds'] for d in arr[:j+1]] for dep in value_dependencies):
            arr[j + 1] = arr[j]
            j -= 1
        
        # Remplazar el valor actual despues del elemento del que dependa
        arr[j + 1] = value
    return list(reversed(arr))

all_dfs = {} # Diccionario que contiene todos los dataframes creados

# Funcion que genera un dataframe a partir de su configuracion
def generate_df(sett):
    result_df = pd.DataFrame()

    # Aqui mandamos a llamar a la funcion calc_greater para determinar cual columna contiene la mayor cantidad de registros
    max_values = calc_greater(sett['columns']) if sett['random'] == False else sett['random_rows']
    
    for col in sett['columns']:
        col_name, col_type = col['name'], col['type']
        
        if col_type == 'category':
            values = col['values']

            if (len(values) != max_values):
                result_df[col_name] = generate_category_random_values(values, max_values)
            else:    
                result_df[col_name] = values

        if col_type == 'date':
            min = col['values']['min']
            max = col['values']['max']
            date_range = pd.date_range(start=min, end=max)
            
            if (date_range.size != max_values):
                result_df[col_name] = generate_date_random_values(min, max, max_values)
            else:    
                result_df[col_name] = pd.Series(date_range)

        if col_type == 'unique':
            result_df[col_name] = generate_unique_random_values(max_values)

        if col_type == 'foreign':
            col_reference_name, col_reference_area = col['values'].split('.')
            
            validate_relationships(all_dfs, col['values']) # Si no se traba aqui por un error de validacion entonces continua su ejecucion

            result_df[col_name] = generate_foreign_random_values(all_dfs[col_reference_name][col_reference_area], max_values)

        if col_type == 'numeric':
            if 'std' in col['values'].keys() and 'mean' in col['values'].keys():
                result_df[col_name] = generate_numeric_random_values(col['values']['min'], col['values']['max'], col['values']['std'], col['values']['mean'], max_values)
            else:
                result_df[col_name] = generate_numeric_random_values(col['values']['min'], col['values']['max'], None, None, max_values)
            
    result_df = pd.DataFrame(result_df)
    result_df.name = sett['ds']

    all_dfs[result_df.name] = result_df
    return result_df

## Main

In [341]:
def build_dataframes(conf_list):
    temp_conf_list = reorder_by_dependencies(conf_list.copy())
    df_arr = []

    for df in temp_conf_list:
        df_arr.append(generate_df(df))

    # Reordenar considerando el orden original (conf_list)
    return  [j for i in conf_list for j in df_arr if i['ds'] == j.name]

## Settings

In [467]:
d1 = {
    "ds": "dataset",
    "columns": [
        {
            "name": "area",
            "type": "category",
            "values": ["TI", "FIN", "HR"]
        },
        {
            "name": "id",
            "type": "unique"
        },
        {
            "name": "Fecha",
            "type": "date",
            "values": {
                "min": "2024-01-01",
                "max": "2024-02-28"
            }
        }
    ],
    "random": False
}

d2 = {
    "ds": "dataset2",
    "columns": [
        {
            "name": "id",
            "type": "unique"
        },
        {
            "name": "area",
            "type": "foreign",
            "values": "dataset.area"
        },
        {
            "name": "subarea",
            "type": "category",
            "values": ["SA1", "SA2", "SA3", "SA4"]
        }
    ],
    "random": False
}

d3 = {
    "ds": "dataset3",
    "columns": [
        {
            "name": "id",
            "type": "unique"
        }, 
        {
            "name": "subarea",
            "type": "foreign",
            "values": "dataset2.id"
        },
        {
            "name": "income",
            "type": "numeric",
            "values": {
                "min": 0,
                "max": 10000
            }
        },
        {
            "name": "goal",
            "type": "numeric",
            "values": {
                "min": 0,
                "max": 50000,
                "std": 20000,
                "mean": 25000
            }
        }
    ],
    "random": True,
    "random_rows": 1000
}

d4 = {
    "ds": "dataset4",
    "columns": [
        {
            "name": "area",
            "type": "category",
            "values": ["TIC", "FIN", "HR", "MKT"]
        },
        {
            "name": "Fecha",
            "type": "date",
            "values": {
                "min": "2024-01-01",
                "max": "2024-02-28"
            }
        },
        {
            "name": "subarea",
            "type": "foreign",
            "values": "dataset3.id"
        },
        {
            "name": "id",
            "type": "unique"
        }
    ],
    "random": False
}

d5 = {
    "ds": "dataset5",
    "columns": [
        {
            "name": "id",
            "type": "unique"
        }, 
        {
            "name": "area",
            "type": "category",
            "values": ["TIC", "FIN", "HR", "MKT"]
        },
        {
            "name": "subarea",
            "type": "category",
            "values": ["SA1", "SA2", "SA3", "SA4"]
        },
        {
            "name": "income",
            "type": "numeric",
            "values": {
                "min": 0,
                "max": 10000
            }
        },
        {
            "name": "goal",
            "type": "numeric",
            "values": {
                "min": 0,
                "max": 50000,
                "std": 20000,
                "mean": 25000
            }
        }
    ],
    "random": True,
    "random_rows": 1000
}


In [471]:
conf_list = [d3, d2, d1, d4, d5]

dataframe_list = build_dataframes(conf_list)

In [473]:
dataframe_list[4]

,id,area,subarea,income,goal
0,279302513d7743f3,MKT,SA3,2426.750,33994.848
1,c1330e22e9134032,FIN,SA2,6299.511,30381.666
2,033516110ee64b62,FIN,SA4,6430.104,29398.033
3,59d49ad49a1a40f9,TIC,SA4,7416.256,5558.124
4,5c941ac8879745be,MKT,SA1,4490.448,36819.805
...,...,...,...,...,...
995,3e963d3a2801439e,FIN,SA3,400.558,44955.417
996,972f6dd4cc4f4761,MKT,SA2,4341.286,14665.296
997,0dcb26797bb4434a,MKT,SA1,3883.098,19219.255
998,19056eff48a4470b,FIN,SA3,2014.303,14110.533


## Requisito #5

In [476]:
# dataframe_list[4] en este caso es el dataframe con random = True
simulation_extended = dataframe_list[4]

In [478]:
simulation_extended

,id,area,subarea,income,goal
0,279302513d7743f3,MKT,SA3,2426.750,33994.848
1,c1330e22e9134032,FIN,SA2,6299.511,30381.666
2,033516110ee64b62,FIN,SA4,6430.104,29398.033
3,59d49ad49a1a40f9,TIC,SA4,7416.256,5558.124
4,5c941ac8879745be,MKT,SA1,4490.448,36819.805
...,...,...,...,...,...
995,3e963d3a2801439e,FIN,SA3,400.558,44955.417
996,972f6dd4cc4f4761,MKT,SA2,4341.286,14665.296
997,0dcb26797bb4434a,MKT,SA1,3883.098,19219.255
998,19056eff48a4470b,FIN,SA3,2014.303,14110.533


In [480]:
# Arreglo de columnas categoricas como numericas del dataframe al cual se le esta realizando el analisis
category_cols = ['area', 'subarea']
numeric_cols = ['income', 'goal']

## Analisis mejor combinacion de variables categoricas 

In [483]:
simulation_extended['n'] = 1
grouped_combination = simulation_extended.groupby(
    category_cols,
    as_index=False
).agg({
    'n': 'count'
})

grouped_combination.columns =  category_cols + ['count']
grouped_combination['prob'] = grouped_combination['count'] / dataframe_list[4].shape[0]

El resultado de este analisis se imprime mas abajo para realizar la comparacion con lo que pasa despues de generar los n cantidad de registros aleatorios

In [486]:
# area_subarea sera el resultado que devuelve la funcion get_categoricas_dataset_simulated
final_simulation = get_categorical_dataset_simulated(simulation_extended, category_cols, 100000)
final_simulation

,area,subarea
2,FIN,SA3
15,TIC,SA4
0,FIN,SA1
10,MKT,SA3
6,HR,SA3
...,...,...
9,MKT,SA2
3,FIN,SA4
8,MKT,SA1
15,TIC,SA4


## Analisis mejor combinacion de variables categoricas despues de generar los n cantidad de registros

In [493]:
final_simulation['n'] = 1
result = final_simulation.groupby(
    category_cols,
    as_index=False,
).agg({
    'n': 'count'
})

result.columns = category_cols + ['count']
result['prob'] = result['count'] / final_simulation.shape[0]

## Comparacion del analisis antes y despues de generar los n cantidad de registros

In [496]:
# Mejor combinacion de variables categoricas desde la primera agrupacion
grouped_combination.sort_values(by='count', ascending=False)

,area,subarea,count,prob
6,HR,SA3,77,0.077
1,FIN,SA2,71,0.071
9,MKT,SA2,70,0.070
3,FIN,SA4,67,0.067
5,HR,SA2,66,0.066
15,TIC,SA4,66,0.066
2,FIN,SA3,64,0.064
8,MKT,SA1,63,0.063
12,TIC,SA1,63,0.063
4,HR,SA1,59,0.059


In [498]:
# Probabilidades de las mejores combinaciones despues de haber generado los 100k registros
result.sort_values(by='count', ascending=False)

,area,subarea,count,prob
6,HR,SA3,7658,0.07658
1,FIN,SA2,6975,0.06975
9,MKT,SA2,6907,0.06907
3,FIN,SA4,6746,0.06746
5,HR,SA2,6686,0.06686
15,TIC,SA4,6680,0.06680
12,TIC,SA1,6462,0.06462
2,FIN,SA3,6351,0.06351
8,MKT,SA1,6235,0.06235
4,HR,SA1,5929,0.05929


Como se puede ver en el ejemplo anterior, las probabilidades despues de haber generado los 100k registros se mantiene "casi" igual como desde la agrupacion original.

In [501]:
# simulated en este caso es una copia de final_simulation excluyendo la variable n que no pertenece al dataframe original
# final_simulation contiene los 100k registros generados aleatoriamente
simulated = final_simulation.drop(columns='n', axis=1).copy()

# Restablece los indices a una secuencia por default antes: [0, 0, 0, ..., n, n] ahora: [0,1,2,3,...,n]
final_simulation = simulated.sort_index().reset_index(drop=True).copy()

# Recorremos el arreglo de columnas numericas
for nc in numeric_cols:
    # Por cada valor numerico, se generan los demas registros aleatoriamente
    # simulated = copia de los 100k registros generados aleatorios
    # simulation_extended = dataframe al cual se le esta realizando el analisis. En este caso este dataframe se creo desde el inicio con 1k registros
    # category_cols = arreglo con las columnas categoricas
    # nc = columna numerica actual a medida se recorre el ciclo de las columnas numericas
    dfn = get_numeric_column_simulated(simulated, simulation_extended, category_cols, nc) 

    # Combina el dataframe actual de final_simulation con el dataframe que se va generando a medida recorra el ciclo
    # Los combina por sus indices (Tomando en cuenta que los indices son unicos)
    final_simulation = pd.merge(
        final_simulation
        , dfn.loc[:,[nc]]
        , left_index=True
        , right_index=True
    )

In [503]:
final_simulation

,area,subarea,income,goal
0,FIN,SA1,3739.363,37271.518
1,FIN,SA1,6057.713,24505.108
2,FIN,SA1,790.043,42825.884
3,FIN,SA1,1388.126,34488.473
4,FIN,SA1,5714.153,14764.042
...,...,...,...,...
99995,TIC,SA4,2327.374,31060.660
99996,TIC,SA4,4258.803,14611.666
99997,TIC,SA4,5939.559,25944.286
99998,TIC,SA4,3518.952,26152.607
